In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, accuracy_score
import numpy as np

# Carica i dati
data = pd.read_excel('cerevisiae_data.xls')

# Separazione delle feature e della classe target
X = data.drop(columns=['Essential', 'orf_id'])
y = data['Essential']

# Ranking CMIM fornito
CMIM_ranking = ['phyletic_retention', 'paralagous_count', 'DIP_degree', 'nucleolus', 'nucleus',
                'upstream_size', 'spindle pole', 'GLU', 'Codon_bias', 'aromaticity_score', 'Gravy_score',
                'CAI', 'downstream_size', 'vacuole', 'endosome', 'mitochondrion', 'PI', 'cytoplasm',
                'promoter_count(Harbison_et_al)', 'vacuolar membrane', 'MET', 'ASP', 'ER to Golgi', 'GLN',
                'peroxisome', 'num places loc', 'LYS', 'ambiguous', 'cell periphery', 'FOP_score', 
                'nuclear periphery', 'ARG', 'Golgi to ER', 'punctate composite', 'ER', 'lipid particle',
                'microtubule', 'Golgi', 'bud neck', 'actin', 'bud', 'Golgi to Vacuole']

# Ordina le colonne di X secondo il ranking CMIM fornito
X_ranked = X[CMIM_ranking]


# Discretizzazione delle feature con il metodo Fayyad & Irani (simulato con KBinsDiscretizer)
# KBinsDiscretizer applica discretizzazione basata sulla frequenza (approssimazione di Fayyad & Irani)
discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')

# Lista per salvare i risultati delle PPV
ppv_scores = []

# Partiamo con tutte le feature e le rimuoviamo una alla volta
for i in range(len(CMIM_ranking), 0, -1):
    # Seleziona le prime 'i' feature del ranking CMIM
    X_selected = X_ranked.iloc[:, :i]
    
    # Discretizza le feature selezionate
    X_discretized = discretizer.fit_transform(X_selected)
    
    # Dividi i dati in training e test set
    X_train, X_test, y_train, y_test = train_test_split(X_discretized, y, test_size=0.5, stratify=y)
    
    # Crea e addestra il classificatore Naive Bayes
    nb_classifier = GaussianNB()
    nb_classifier.fit(X_train, y_train)
    
    # Fai le predizioni sul test set
    y_pred = nb_classifier.predict(X_test)
    
    # Calcola la PPV (precision score) per le predizioni positive
    ppv_top_5 = precision_score(y_test, y_pred, average='binary')
    
    # Salva il risultato (numero di feature e PPV)
    ppv_scores.append((i, ppv_top_5))

# Identifica il numero ottimale di feature
best_num_features = max(ppv_scores, key=lambda x: x[1])[0]
print(f'Numero ottimale di feature: {best_num_features}')

# Usa il numero ottimale di feature per la classificazione finale
X_optimal = X_ranked.iloc[:, :best_num_features]

# Discretizza le feature ottimali
X_discretized_opt = discretizer.fit_transform(X_optimal)

# Divisione in train e test set
X_train_opt, X_test_opt, y_train_opt, y_test_opt = train_test_split(X_discretized_opt, y, test_size=0.5, stratify=y)
# Addestra il classificatore finale con le feature ottimali
nb_classifier_opt = GaussianNB()
nb_classifier_opt.fit(X_train_opt, y_train_opt)

# Fai predizioni finali e valuta l'accuratezza
y_pred_opt = nb_classifier_opt.predict(X_test_opt)
final_accuracy = accuracy_score(y_test_opt, y_pred_opt)
print(f'Final accuracy with {best_num_features} features: {final_accuracy}')

# Bootstrap per valutare la stabilità dell'accuratezza
n_iterations = 100
bootstrap_accuracies = []

for _ in range(n_iterations):
    # Campionamento con rimpiazzo
    indices = np.random.choice(len(X_train_opt), len(X_train_opt), replace=True)
    X_train_bootstrap = X_train_opt[indices]
    y_train_bootstrap = y_train_opt.iloc[indices]
    
    # Addestra il classificatore con il campione bootstrap
    nb_classifier_bootstrap = GaussianNB()
    nb_classifier_bootstrap.fit(X_train_bootstrap, y_train_bootstrap)
    
    # Fai predizioni sul test set
    y_pred_bootstrap = nb_classifier_bootstrap.predict(X_test_opt)
    
    # Calcola l'accuratezza
    bootstrap_accuracy = accuracy_score(y_test_opt, y_pred_bootstrap)
    bootstrap_accuracies.append(bootstrap_accuracy)

# Calcola la media e l'intervallo di confidenza delle accuratezze bootstrap
mean_accuracy = np.mean(bootstrap_accuracies)
conf_interval = np.percentile(bootstrap_accuracies, [2.5, 97.5])

print(f'Mean bootstrap accuracy: {mean_accuracy}')
print(f'95% confidence interval for accuracy: {conf_interval}')


Numero ottimale di feature: 6
Final accuracy with 6 features: 0.8028764805414551
Mean bootstrap accuracy: 0.8015693739424704
95% confidence interval for accuracy: [0.78600888 0.81790398]
